<a href="https://colab.research.google.com/github/balakhnyov/technoatom-ML-homeworks/blob/main/homework8/HW6_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: ignored

In [ ]:
spark

*`ngrok может отработать не с первого раза, повторите при необходимости.`*

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-06-02 18:02:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.204.235.80, 34.233.212.111, 3.211.204.50, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.204.235.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  52.6MB/s    in 0.3s    

2021-06-02 18:02:01 (52.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://9717536701d3.ngrok.io


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
ss = SparkSession(sc)

In [ ]:
wp = ss.read.csv("War and Peace by Leo Tolstoy (ru).txt", sep='%s\t%s', encoding='windows-1251', header='true', inferSchema='true',)

In [ ]:
wp.show(10)

+--------------------+
|Война и мир. Книга 1|
+--------------------+
|Лев Николаевич То...|
|      Война и мир #1|
|В книгу вошли пер...|
|          Том первый|
|        Часть первая|
|                   I|
|– Eh bien, mon pr...|
|Так говорила в ию...|
|«Si vous n’avez r...|
|– Dieu, quelle vi...|
+--------------------+
only showing top 10 rows



In [ ]:
wp.filter(length(col("`Война и мир. Книга 1`")) > 0).withColumn("words", split(col("`Война и мир. Книга 1`"), "\\s+")).select(explode(col("words"))).show(20)

+----------+
|       col|
+----------+
|       Лев|
|Николаевич|
|   Толстой|
|     Война|
|         и|
|       мир|
|        #1|
|         В|
|     книгу|
|     вошли|
|    первый|
|         и|
|    второй|
|      тома|
|    романа|
|    «Война|
|         и|
|      мир»|
|         –|
|    одного|
+----------+
only showing top 20 rows



In [ ]:
wp.filter(length(col("`Война и мир. Книга 1`")) > 0).withColumn("words", split(col("`Война и мир. Книга 1`"), "\\s+")).select(explode(col("words"))).describe().show()

+-------+------------------+
|summary|               col|
+-------+------------------+
|  count|            475218|
|   mean| 320.5295652173913|
| stddev|435.44152087066476|
|    min|                  |
|    max|            № 178.|
+-------+------------------+



In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

In [ ]:
training = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-train.txt")
test = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-test.txt")

In [ ]:
sidxr = feature.StringIndexer(inputCol="label", outputCol="indxLabel").fit(training)

In [ ]:
vidxr = feature.VectorIndexer(inputCol="features", outputCol="inxFeatures", handleInvalid='skip').fit(training)

In [ ]:
decision_tree = classification.DecisionTreeClassifier(labelCol="indxLabel", featuresCol="inxFeatures")

In [ ]:
pipeline = Pipeline(stages=[sidxr, vidxr, decision_tree])

In [ ]:
model = pipeline.fit(training)

In [ ]:
prediction= model.transform(test)

In [ ]:
eval = evaluation.MulticlassClassificationEvaluator(labelCol="indxLabel", predictionCol="prediction", metricName="accuracy")
accuracy = eval.evaluate(prediction)
accuracy

0.6793041029766693

Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

Тут лежат данные для решения задач на sql. Необходимо написать код который формирует из данных датафреймы и используя sqlContext написать sql код с решением. Приветствуется наличие нескольких вариантов решения задачи.

In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import unix_timestamp, from_unixtime
sqlContext = SQLContext(sc)

In [ ]:
regions = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/regions.csv')
regions.registerTempTable("regions")
regions.show()

+---------+--------------------+
|REGION_ID|         REGION_NAME|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [ ]:
locations = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/locations.csv')
locations.registerTempTable("locations")
locations.show()

+-----------+--------------------+-----------+-------------------+----------------+----------+
|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|               CITY|  STATE_PROVINCE|COUNTRY_ID|
+-----------+--------------------+-----------+-------------------+----------------+----------+
|       1000|1297 Via Cola di Rie|      00989|               Roma|            null|        IT|
|       1100|93091 Calle della...|      10934|             Venice|            null|        IT|
|       1200|    2017 Shinjuku-ku|       1689|              Tokyo|Tokyo Prefecture|        JP|
|       1300|     9450 Kamiya-cho|       6823|          Hiroshima|            null|        JP|
|       1400| 2014 Jabberwocky Rd|      26192|          Southlake|           Texas|        US|
|       1500| 2011 Interiors Blvd|      99236|South San Francisco|      California|        US|
|       1600|      2007 Zagora St|      50090|    South Brunswick|      New Jersey|        US|
|       1700|     2004 Charade Rd|      98199|    

In [ ]:
jobs = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/jobs.csv')
jobs.registerTempTable("jobs")
jobs.show()

+----------+--------------------+----------+----------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+----------+--------------------+----------+----------+
|   AD_PRES|           President|     20080|     40000|
|     AD_VP|Administration Vi...|     15000|     30000|
|   AD_ASST|Administration As...|      3000|      6000|
|    FI_MGR|     Finance Manager|      8200|     16000|
|FI_ACCOUNT|          Accountant|      4200|      9000|
|    AC_MGR|  Accounting Manager|      8200|     16000|
|AC_ACCOUNT|   Public Accountant|      4200|      9000|
|    SA_MAN|       Sales Manager|     10000|     20080|
|    SA_REP|Sales Representative|      6000|     12008|
|    PU_MAN|  Purchasing Manager|      8000|     15000|
|  PU_CLERK|    Purchasing Clerk|      2500|      5500|
|    ST_MAN|       Stock Manager|      5500|      8500|
|  ST_CLERK|         Stock Clerk|      2008|      5000|
|  SH_CLERK|      Shipping Clerk|      2500|      5500|
|   IT_PROG|          Programmer|      4000|    

In [ ]:
job_history = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/job_history.csv')
job_history.registerTempTable("job_history")
job_history.show()

+-----------+----------+--------+----------+-------------+
|EMPLOYEE_ID|START_DATE|END_DATE|    JOB_ID|DEPARTMENT_ID|
+-----------+----------+--------+----------+-------------+
|        102|  13.01.01|24.07.06|   IT_PROG|           60|
|        101|  21.09.97|27.10.01|AC_ACCOUNT|          110|
|        101|  28.10.01|15.03.05|    AC_MGR|          110|
|        201|  17.02.04|19.12.07|    MK_REP|           20|
|        114|  24.03.06|31.12.07|  ST_CLERK|           50|
|        122|  01.01.07|31.12.07|  ST_CLERK|           50|
|        200|  17.09.95|17.06.01|   AD_ASST|           90|
|        176|  24.03.06|31.12.06|    SA_REP|           80|
|        176|  01.01.07|31.12.07|    SA_MAN|           80|
|        200|  01.07.02|31.12.06|AC_ACCOUNT|           90|
+-----------+----------+--------+----------+-------------+



In [ ]:
employees = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/employees.csv')
employees.registerTempTable("employees")
employees.show()

In [ ]:
departments = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/departments.csv')
departments.registerTempTable("departments")
departments.show(10)

+-------------+----------------+----------+-----------+
|DEPARTMENT_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+----------------+----------+-----------+
|           10|  Administration|       200|       1700|
|           20|       Marketing|       201|       1800|
|           30|      Purchasing|       114|       1700|
|           40| Human Resources|       203|       2400|
|           50|        Shipping|       121|       1500|
|           60|              IT|       103|       1400|
|           70|Public Relations|       204|       2700|
|           80|           Sales|       145|       2500|
|           90|       Executive|       100|       1700|
|          100|         Finance|       108|       1700|
+-------------+----------------+----------+-----------+
only showing top 10 rows



In [ ]:
country = sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load('/content/data/country.csv')
country.registerTempTable("country")
country.show()

+----------+------------+---------+
|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
|        BE|     Belgium|        1|
|        BR|      Brazil|        2|
|        CA|      Canada|        2|
|        CH| Switzerland|        1|
|        CN|       China|        3|
|        DE|     Germany|        1|
|        DK|     Denmark|        1|
|        EG|       Egypt|        4|
|        FR|      France|        1|
|        IL|      Israel|        4|
|        IN|       India|        3|
|        IT|       Italy|        1|
|        JP|       Japan|        3|
|        KW|      Kuwait|        4|
|        ML|    Malaysia|        3|
|        MX|      Mexico|        2|
|        NG|     Nigeria|        4|
|        NL| Netherlands|        1|
+----------+------------+---------+
only showing top 20 rows



Кто получает больше всего? Кто меньше всего?

Выведете топ 5 по зарплате.

In [ ]:
employees = employees.withColumn("SALARY", employees["SALARY"].cast(IntegerType()))

In [ ]:
employees.registerTempTable("employees")

In [ ]:
sqlContext.sql("""
SELECT * FROM employees
  WHERE SALARY = (SELECT MAX(SALARY) FROM employees)
""").show()

+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|SKING|515.123.4567| 17.06.03|AD_PRES| 24000|          null|      null|           90|
+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+



In [ ]:
sqlContext.sql("""
SELECT FIRST_NAME, LAST_NAME FROM, SALARY FROM employees
  WHERE SALARY = (SELECT MIN(SALARY) FROM employees)
""").show()

+----------+-----+------+
|FIRST_NAME| FROM|SALARY|
+----------+-----+------+
|        TJ|Olson|  2100|
+----------+-----+------+



In [ ]:
sqlContext.sql("""
SELECT FIRST_NAME, LAST_NAME, SALARY, row_number() over(order by SALARY DESC) as rate FROM employees LIMIT 5
""").show()

+----------+---------+------+----+
|FIRST_NAME|LAST_NAME|SALARY|rate|
+----------+---------+------+----+
|    Steven|     King| 24000|   1|
|     Neena|  Kochhar| 17000|   2|
|       Lex|  De Haan| 17000|   3|
|      John|  Russell| 14000|   4|
|     Karen| Partners| 13500|   5|
+----------+---------+------+----+



Сколько всего регионов? Сколько работников в каждом регионе?

In [ ]:
sqlContext.sql("SELECT * FROM regions").show()

+---------+--------------------+
|REGION_ID|         REGION_NAME|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [ ]:
sqlContext.sql("""
SELECT count(EMPLOYEE_ID), REGION_ID FROM employees
  WHERE SALARY = (SELECT min(salary) FROM employees)
""").show()

+------------------+---------+
|count(EMPLOYEE_ID)|REGION_ID|
+------------------+---------+
|                36|        1|
|                70|        2|
|                 0|        3|
|                 0|        4|
+------------------+---------+



Выведете всех работников из Китая.

In [ ]:
sqlContext.sql("""
SELECT * FROM employees
  FULL JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
    FULL JOIN locations ON locations.LOCATION_ID = departments.LOCATION_ID
      FULL JOIN country ON country.COUNTRY_ID = locations.COUNTRY_ID
        WHERE locations.COUNTRY_ID = 'CN'
""").show()

+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+-------------+---------------+----------+-----------+-----------+-------------------+-----------+-------+--------------+----------+----------+------------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE|JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|LOCATION_ID|     STREET_ADDRESS|POSTAL_CODE|   CITY|STATE_PROVINCE|COUNTRY_ID|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+-------------+---------------+----------+-----------+-----------+-------------------+-----------+-------+--------------+----------+----------+------------+---------+
|       null|      null|     null| null|        null|     null|  null|  null|          null|      null|         null|         null|           null|    

Укажите самую высокооплачиваемою должность.

In [ ]:
jobs = jobs.withColumn("MAX_SALARY", jobs["MAX_SALARY"].cast(IntegerType()))
jobs.registerTempTable("jobs")

In [ ]:
sqlContext.sql("""
SELECT JOB_TITLE, MAX_SALARY FROM jobs
  WHERE MAX_SALARY = (SELECT MAX(MAX_SALARY) FROM jobs)
""").show()

+---------+----------+
|JOB_TITLE|MAX_SALARY|
+---------+----------+
|President|     40000|
+---------+----------+



Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
sqlContext.sql("""
SELECT FIRST_NAME, LAST_NAME, MANAGER_ID FROM employees
    WHERE JOB_ID = 'IT_PROG'
""").show()

+----------+---------+----------+
|FIRST_NAME|LAST_NAME|MANAGER_ID|
+----------+---------+----------+
| Alexander|   Hunold|       102|
|     Bruce|    Ernst|       103|
|     David|   Austin|       103|
|     Valli|Pataballa|       103|
|     Diana|  Lorentz|       103|
+----------+---------+----------+



In [ ]:
sqlContext.sql("""
SELECT FIRST_NAME, LAST_NAME, employees.MANAGER_ID FROM employees
  INNER JOIN departments ON employees.DEPARTMENT_ID = departments.DEPARTMENT_ID
    WHERE DEPARTMENT_NAME = 'IT'
""").show()

+----------+---------+----------+
|FIRST_NAME|LAST_NAME|MANAGER_ID|
+----------+---------+----------+
| Alexander|   Hunold|       102|
|     Bruce|    Ernst|       103|
|     David|   Austin|       103|
|     Valli|Pataballa|       103|
|     Diana|  Lorentz|       103|
+----------+---------+----------+



Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [ ]:
table = sqlContext.sql("""
SELECT FIRST_NAME, LAST_NAME, employees.JOB_ID as JOB_ID_NOW,
job_history.JOB_ID, START_DATE, END_DATE, HIRE_DATE FROM job_history
    RIGHT JOIN employees ON employees.EMPLOYEE_ID = job_history.EMPLOYEE_ID 
""")

In [ ]:
table.show()

+-----------+---------+----------+----------+----------+--------+---------+
| FIRST_NAME|LAST_NAME|JOB_ID_NOW|    JOB_ID|START_DATE|END_DATE|HIRE_DATE|
+-----------+---------+----------+----------+----------+--------+---------+
|     Steven|     King|   AD_PRES|      null|      null|    null| 17.06.03|
|      Neena|  Kochhar|     AD_VP|    AC_MGR|  28.10.01|15.03.05| 21.09.05|
|      Neena|  Kochhar|     AD_VP|AC_ACCOUNT|  21.09.97|27.10.01| 21.09.05|
|        Lex|  De Haan|     AD_VP|   IT_PROG|  13.01.01|24.07.06| 13.01.01|
|  Alexander|   Hunold|   IT_PROG|      null|      null|    null| 03.01.06|
|      Bruce|    Ernst|   IT_PROG|      null|      null|    null| 21.05.07|
|      David|   Austin|   IT_PROG|      null|      null|    null| 25.06.05|
|      Valli|Pataballa|   IT_PROG|      null|      null|    null| 05.02.06|
|      Diana|  Lorentz|   IT_PROG|      null|      null|    null| 07.02.07|
|      Nancy|Greenberg|    FI_MGR|      null|      null|    null| 17.08.02|
|     Daniel

In [ ]:
table_formatted = table\
.withColumn('START_DATE-formatted',when((table.START_DATE.isNull() | (table.START_DATE == '')) ,'0')\
.otherwise(from_unixtime(unix_timestamp(table.START_DATE,'dd.MM.yy'))))\
.withColumn('END_DATE-formatted',when((table.END_DATE.isNull() | (table.END_DATE == '')) ,'0')\
.otherwise(from_unixtime(unix_timestamp(table.END_DATE,'dd.MM.yy'))))\
.withColumn('HIRE_DATE-formatted',when((table.HIRE_DATE.isNull() | (table.HIRE_DATE == '')) ,'0')\
.otherwise(from_unixtime(unix_timestamp(table.HIRE_DATE,'dd.MM.yy'))))\
.drop('START_DATE','END_DATE', 'HIRE_DATE')

In [ ]:
table_formatted.show(2)

+----------+---------+----------+------+--------------------+-------------------+-------------------+
|FIRST_NAME|LAST_NAME|JOB_ID_NOW|JOB_ID|START_DATE-formatted| END_DATE-formatted|HIRE_DATE-formatted|
+----------+---------+----------+------+--------------------+-------------------+-------------------+
|    Steven|     King|   AD_PRES|  null|                   0|                  0|2003-06-17 00:00:00|
|     Neena|  Kochhar|     AD_VP|AC_MGR| 2001-10-28 00:00:00|2005-03-15 00:00:00|2005-09-21 00:00:00|
+----------+---------+----------+------+--------------------+-------------------+-------------------+
only showing top 2 rows



Выведете уникальные телефонные номера

In [ ]:
sqlContext.sql("""
SELECT DISTINCT PHONE_NUMBER FROM employees
""").show()

+------------------+
|      PHONE_NUMBER|
+------------------+
|011.44.1344.429018|
|      515.127.4566|
|      515.127.4564|
|011.44.1344.429278|
|      515.123.4569|
|      650.124.1434|
|      650.123.2234|
|011.44.1344.498718|
|      650.127.1634|
|      515.127.4561|
|011.44.1345.629268|
|      515.127.4562|
|011.44.1644.429264|
|011.44.1644.429262|
|      650.501.1876|
|      650.127.1834|
|011.44.1343.529268|
|011.44.1644.429265|
|      515.123.8181|
|      650.507.9833|
+------------------+
only showing top 20 rows



Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
sqlContext.sql("""
SELECT LAST_NAME, count(LAST_NAME) FROM employees
  GROUP BY LAST_NAME HAVING count(LAST_NAME) > 1
""").show()

+---------+----------------+
|LAST_NAME|count(LAST_NAME)|
+---------+----------------+
|    Smith|               2|
|     King|               2|
|Cambrault|               2|
|   Taylor|               2|
+---------+----------------+

